In [ ]:
# Mechanism to mock test submits in MongoDB with random dates.
from bson import ObjectId
from config.database import db_UJ        # UJ mongoDB
from datetime import datetime, timedelta
from pymongo import UpdateOne
import random

class MockTestSubmits:
    def __init__(self):
        self.cts = db_UJ['test_st']

    def random_date_gen(self, simplify=False, total_dates=5):
        def df(dt):
            return dt.strftime("%m-%d")

        now = datetime.now()  # Get current date and time
        dates = []

        # Generate dates for the last 3 days (today and the two previous days)
        for i in range(3):
            date = now - timedelta(days=i)
            dates.append(date)

        # Randomly select 5 items from these dates (with replacement)
        random_items = random.choices(dates, k=random.randint(1, total_dates))
        # If simplify is True, format the dates
        if simplify:
            # Format the dates using your df function
            formatted_items = [df(item) for item in random_items]
            return formatted_items    
        else:
            return random_items
        # Format the dates using your df function
        # formatted_items = [df(item) for item in random_items]
        # return formatted_items

    def update_operations(self,mdb_set):
        cts = db_UJ['test_st']   # create/get a collection
        doc_ids = list(cts.find({},{"_id":1}))       # Get limited test document IDs
        update_operation = []
        for i in doc_ids:
            juice = {'submits':self.random_date_gen(simplify=False, total_dates=15)}
            # Use this to remove all the submits fields
            # Use this to simulate setting the new submits field with mock timestamps
            set_ops = {'$set': juice} if mdb_set else {'$unset': {'submits': ''}}
            update_operation.append(
                UpdateOne(
                {"_id": ObjectId(i['_id'])},
                set_ops
            ))
        return update_operation

    def update_submits(self, mdb_set):
        self.cts.bulk_write(self.update_operations(mdb_set=mdb_set))

    def check_transformed_submits(self):
        original = list(self.cts.find({'submits': {'$exists': True}},{"_id":0,"ph":0,"r_id":0}))
        transformed = [{v1: v2} for d in original for v1, v2 in zip(d.values(), list(d.values())[1:])]
        return transformed

mts = MockTestSubmits()
# mts.update_operations(mdb_set=False)
# mts.update_submits(mdb_set=False)

In [12]:
import re

from routes.route import flight_aware_w_auth
n_pattern = re.compile("^N[a-zA-Z0-9]{5}$")
n_pattern.match('N12377')

# flight_aware_w_auth()

c:\Users\ujasv\OneDrive\Desktop\base\cirrostrats-backend\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<re.Match object; span=(0, 6), match='N12377'>

In [1]:
from routes.route import get_search_suggestions


# x = await get_search_suggestions(email='', query='pa', limit=1000)


c:\Users\ujasv\OneDrive\Desktop\base\cirrostrats-backend\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
import itertools
import string

# Generate all combinations of length 2
alphabets = string.ascii_lowercase[:5]
combinations = [''.join(c) for c in itertools.combinations(alphabets, 2)]
permutations = [''.join(p) for p in itertools.permutations(alphabets, 2)]

print(combinations)
print(f"Total combinations: {len(combinations)}", f"Total permut: {len(permutations)}")

['ab', 'ac', 'ad', 'ae', 'bc', 'bd', 'be', 'cd', 'ce', 'de']
Total combinations: 10 Total permut: 20


In [ ]:
a = ctrs.aggregate([
    { "$match": { "submits": { "$exists": True} } },
    { "$unwind": "$submits" },
    { "$project": {
        "_id": 0,
        "rst": 1,
        "timestamp": "$submits"
    }}
])
# list(a)
ctrs.find_one({})


In [ ]:
cts_returns =  list(mts.cts.aggregate([
        { "$match": { "submits": { "$exists": True} } },
        { "$unwind": "$submits" },
        { "$addFields": { "timestamp": "$submits" } },
        { "$unset": ["_id", "r_id", "ph", "submits"] }
    ]))

In [ ]:
list(mts.cts.find({"submits": {"$exists": True}}))